In [ ]:
import os 
os.environ["CUDA_VISIBLE_DEVICES"] = "2,3"
import sys
import glob
import argparse
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from keras.applications.resnet50 import ResNet50 #, preprocess_inputcond
from keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg16 import VGG16
from keras.applications.inception_resnet_v2 import InceptionResNetV2

from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator,array_to_img, img_to_array, load_img
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint,LearningRateScheduler
import pandas as pd
#from data import *

from keras.utils import to_categorical
from keras.utils import multi_gpu_model

In [2]:
IM_WIDTH, IM_HEIGHT = 256, 256
NB_EPOCHS = 3
BAT_SIZE = 32
FC_SIZE = 1024
NB_IV3_LAYERS_TO_FREEZE = 0

In [4]:
class dataProcess(object):

    def __init__(self, out_rows, out_cols, image_path="../data/image/", train1_path = "/train/A", train2_path = "/train/P1B", test1_path = "/test/A",test2_path = "/test/P1B", npy_path = "../npy/", img_type = "jpg"):

        """

        """

        self.out_rows = out_rows
        self.out_cols = out_cols
        self.image_path = image_path
        self.train1_path = train1_path
        self.train2_path = train2_path
        self.img_type = img_type
        self.test1_path = test1_path
        self.test2_path = test2_path
        self.npy_path = npy_path

    def create_train_data(self, index):
        npy_folder_path = self.npy_path+'%s'%str(index)

        i = 0
        print('-'*30)
        print('Creating training images...')
        print('-'*30)
        train1_folder_path = self.image_path+'%s'%str(index)+ self.train1_path
        imgs1 = glob.glob(train1_folder_path+"/*."+self.img_type)
        print(train1_folder_path)
        print(len(imgs1))


        imgdatas1 = np.ndarray((len(imgs1),self.out_rows,self.out_cols,3), dtype=np.uint8)
        imglabels1 = np.ndarray((len(imgs1),1), dtype=np.uint8)
        for imgname in imgs1:
            midname = imgname[imgname.rindex("/")+1:]

            img = load_img(train1_folder_path + "/" + midname)
            img=img.resize((256,256))
            img = img_to_array(img)
            imgdatas1[i] = img
            imglabels1[i] = 0
            if i % 100 == 0:
                print('Done: {0}/{1} images'.format(i, len(imgs1)))
            i += 1
        print('loading done')

        np.save(npy_folder_path + '/train1_256.npy', imgdatas1)
        np.save(npy_folder_path + '/label1_256.npy', imglabels1)
        print('Saving to .npy files done. (1)')

        i = 0
        train2_folder_path = self.image_path+'%s'%str(index)+ self.train2_path
        imgs2 = glob.glob(train2_folder_path+"/*."+self.img_type)
        print(len(imgs2))
        imgdatas2 = np.ndarray((len(imgs2),self.out_rows,self.out_cols,3), dtype=np.uint8)
        imglabels2 = np.ndarray((len(imgs2),1), dtype=np.uint8)
        for imgname in imgs2:
            midname = imgname[imgname.rindex("/")+1:]

            img2 = load_img(train2_folder_path + "/" + midname)
            img2=img2.resize((256,256))
            img2 = img_to_array(img2)
            imgdatas2[i] = img2
            imglabels2[i] = 1
            if i % 100 == 0:
                print('Done: {0}/{1} images'.format(i, len(imgs2)))
            i += 1
        print('loading done')
        np.save(npy_folder_path + '/train2_256.npy', imgdatas2)
        np.save(npy_folder_path + '/label2_256.npy', imglabels2)
        print('Saving to .npy files done. (2)')

    def create_test_data(self, index):
        npy_folder_path = self.npy_path+'%s'%str(index)
        i = 0
        print('-'*30)
        print('Creating test images...')
        print('-'*30)

        test1_folder_path = self.image_path+'%s'%str(index)+ self.test1_path
        test2_folder_path = self.image_path+'%s'%str(index)+ self.test2_path
        imgs1 = glob.glob(test1_folder_path+"/*."+self.img_type)
        imgs2 = glob.glob(test2_folder_path+"/*."+self.img_type)
        print(len(imgs1))
        print(len(imgs2))

        imgdatas1 = np.ndarray((len(imgs1),self.out_rows,self.out_cols,3), dtype=np.uint8)
        imglabels1 = np.ndarray((len(imgs1),1), dtype=np.uint8)
        for imgname in imgs1:
            midname = imgname[imgname.rindex("/")+1:]
            img = load_img(test1_folder_path + "/" + midname)	
            img=img.resize((256,256))
            img = img_to_array(img)
            imgdatas1[i] = img
            imglabels1[i] = 0
            i += 1
        print('loading done')
        np.save(npy_folder_path + '/test1_256.npy', imgdatas1)
        np.save(npy_folder_path + '/test_label1_256.npy', imglabels1)
        print('Saving to imgs_test.npy files done.(1)')

        i = 0
        imgdatas2 = np.ndarray((len(imgs2),self.out_rows,self.out_cols,3), dtype=np.uint8)
        imglabels2 = np.ndarray((len(imgs2),1), dtype=np.uint8)
        for imgname in imgs2:
            midname = imgname[imgname.rindex("/")+1:]
            img = load_img(test2_folder_path + "/" + midname)	
            img=img.resize((256,256))
            img = img_to_array(img)
            imgdatas2[i] = img
            imglabels2[i] = 1
            i += 1
        print('loading done')
        np.save(npy_folder_path + '/test2_256.npy', imgdatas2)
        np.save(npy_folder_path + '/test_label2_256.npy', imglabels2)
        print('Saving to imgs_test.npy files done.(1)')


    def load_train_data(self, index):
        npy_folder_path = self.npy_path+'%s'%str(index)
        print(npy_folder_path)
        
        print('-'*30)
        print('load train images...')
        print('-'*30)
        
        imgs_train1 = np.load(npy_folder_path+"/train1_256.npy")
        imgs_mask_train1 = np.load(npy_folder_path+"/label1_256.npy")
        imgs_train1 = imgs_train1.astype('float32')/255

        imgs_train2 = np.load(npy_folder_path+"/train2_256.npy")
        imgs_mask_train2 = np.load(npy_folder_path+"/label2_256.npy")  
        imgs_train2 = imgs_train2.astype('float32')/255
        
        imgs_train=np.append(imgs_train1,imgs_train2,axis=0)
        imgs_mask_train=np.append(imgs_mask_train1,imgs_mask_train2,axis=0)
        print(imgs_train.shape)
        print(imgs_mask_train.shape)



        return imgs_train,imgs_mask_train

    def load_test_data(self, index):
        npy_folder_path = self.npy_path+'%s'%str(index)
        print(npy_folder_path)
        
        print('-'*30)
        print('load test images...')
        print('-'*30)
        imgs_test1 = np.load(npy_folder_path+"/test1_256.npy")
        imgs_mask_test1 = np.load(npy_folder_path+"/test_label1_256.npy")
        imgs_test1 = imgs_test1.astype('float32')/255

        imgs_test2 = np.load(npy_folder_path+"/test2_256.npy")
        imgs_mask_test2 = np.load(npy_folder_path+"/test_label2_256.npy")
        imgs_test2 = imgs_test2.astype('float32')/255
        
        imgs_test=np.append(imgs_test1,imgs_test2,axis=0)
        imgs_mask_test=np.append(imgs_mask_test1,imgs_mask_test2,axis=0)
        print(imgs_test.shape)
        print(imgs_mask_test.shape)



        return imgs_test,imgs_mask_test

In [ ]:
# create train, test data
if __name__ == "__main__":

    mydata = dataProcess(256,256)
    mydata.create_train_data(0)
    mydata.create_train_data(1)
    mydata.create_train_data(2)
    mydata.create_train_data(3)
    mydata.create_train_data(4)

    mydata.create_test_data(0)
    mydata.create_test_data(1)
    mydata.create_test_data(2)
    mydata.create_test_data(3)
    mydata.create_test_data(4)

In [ ]:
def sch(epoch):
    if epoch>100:
        return 0.00001
    else:
        return 0.0001
    
def get_nb_files(directory):
    if not os.path.exists(directory):
        return 0
    cnt = 0
    for r, dirs, files in os.walk(directory):
        for dr in dirs:
            cnt += len(glob.glob(os.path.join(r, dr + "/*")))
    return cnt

def add_new_last_layer(base_model, nb_classes):
    x = base_model.layers[-2].output  
    x = GlobalAveragePooling2D()(x)
    #x = Dense(FC_SIZE, activation='relu')(x)
    predictions = Dense(nb_classes, activation='softmax')(x)
    model = Model(input=base_model.input, output=predictions)

    return model

def setup_to_finetune(model):
    for layer in model.layers[:NB_IV3_LAYERS_TO_FREEZE]:
        layer.trainable = False
    for layer in model.layers[NB_IV3_LAYERS_TO_FREEZE:]:
        layer.trainable = True
    model=multi_gpu_model(model,gpus=2)
    model.compile(optimizer=SGD(lr=0.0001,decay=1e-6, momentum=0.9,nesterov=True), loss='categorical_crossentropy', metrics=['accuracy'])

def train():
    for index in range(1,5):
        mydata = dataProcess(256, 256)
        X_train,y_train=mydata.load_train_data(index)
        X_test,y_test=mydata.load_test_data(index)
        print(y_train.shape)
        print(y_test.shape)
        y_train=to_categorical(y_train,2)
        y_test=to_categorical(y_test,2)

        # setup model
        base_model = ResNet50(weights='imagenet', include_top=False)
        model = add_new_last_layer(base_model, 2)
        model.summary()
        #sc=LearningRateScheduler(sch)
        for layer in model.layers[:NB_IV3_LAYERS_TO_FREEZE]:
            layer.trainable = False
        for layer in model.layers[NB_IV3_LAYERS_TO_FREEZE:]:
            layer.trainable = True

        model=multi_gpu_model(model,gpus=2)
        model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

        model_checkpoint = ModelCheckpoint('model_normalize_best(cv%s).h5'%str(index), monitor='val_loss',mode='min', verbose=1, save_best_only=True,save_weights_only=False)
        sc=LearningRateScheduler(sch)
        model.fit(X_train, y_train, batch_size=40, epochs=300, verbose=1,validation_split=0.2, shuffle=True,callbacks=[model_checkpoint,sc])#1000

        model.save("../result/model/model_normalize_last(cv%s).h5"%str(index))

        print('predict test data')
        imgs_mask_test = model.predict(X_test, batch_size=1, verbose=1)
        np.save('../result/npy/test_pred(cv%s).npy'%str(index), imgs_mask_test)
       
        
#python classification.py --all_dir ../--nb_epoch 300 --batch_size 8

if __name__=="__main__":
    train()

../npy/1
------------------------------
load train images...
------------------------------
(732, 256, 256, 3)
(732, 1)
../npy/1
------------------------------
load test images...
------------------------------
(183, 256, 256, 3)
(183, 1)
(732, 1)
(183, 1)


/home/ubuntu/.conda/envs/younji/lib/python3.6/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '
/home/ubuntu/.conda/envs/younji/lib/python3.6/site-packages/ipykernel_launcher.py:21: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, None, None, 6 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, None, None, 6 256         conv1[0][0]                      
____________________________________________________________________________________________

In [ ]:
from keras.models import load_model

model_filename = '../../exp_CLAHE_test/result/model/model_normalize_last(cv0).h5'
outer_keras_model = load_model(model_filename)
outer_keras_model.summary()

In [12]:
model.save("/home/ubuntu/gcubme4/Workspace/YJ_Kim/exp_CLAHE/code/model/model_normalize_last(cv%s).h5"%str(0))

print('predict test data')
imgs_mask_test = model.predict(X_test, batch_size=1, verbose=1)
np.save('/home/ubuntu/gcubme4/Workspace/YJ_Kim/exp_CLAHE/result/npy/test_pred(cv%s).npy'%str(0), imgs_mask_test)

predict test data
936/936 [==============================] - 18s 20ms/step


In [6]:
# -*- coding: utf-8 -*-
"""
Created on Wed Feb 21 13:57:30 2018

@author: gachon
"""
import os 
os.environ["CUDA_VISIBLE_DEVICES"] = "2,3"

import numpy as np
import pandas as pd
#from data import *
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score,precision_score,f1_score,recall_score,roc_curve,auc
from keras.preprocessing.image import array_to_img
from keras.models import Model, load_model

cv_cnt=0

#for cv_cnt in range(5):
mydata = dataProcess(256, 256)
X_test,y_test = mydata.load_test_data(cv_cnt)

print('-'*30)
print('load model...')
print('-'*30)

model = load_model('../result/model/model_normalize_last(cv%s).h5'%str(cv_cnt))
y_pred = model.predict(X_test, batch_size=1, verbose=1)
#np.save('../result/%s/npy/test_pred_epoch300(cv%s).npy'%(str(cv_cnt), str(cv_cnt)), y_pred)


# f=open("../../exp_sharp/result_edit/%s/txt/result(cv%s).txt"%(str(cv_cnt),str(cv_cnt)),"w")
pred_label = []
for i, v in enumerate(y_pred):
	pre_ans = v.argmax()
	pred_label.append(pre_ans)
# 	txt = str(y_test[i])+","+str(pre_ans)+","+str(v[0])+","+str(v[1])+"\n"
# 	f.write(txt)
# f.close()

'''
pred_label = []
for i, v in enumerate(y_pred):
	pre_ans = v.argmax()
	pred_label.append(pre_ans)
'''

cm = confusion_matrix(y_test,pred_label)
print(cm)
acc = accuracy_score(y_test,pred_label)
precision = precision_score(y_test,pred_label)
f1score = f1_score(y_test,pred_label)
recall = recall_score(y_test,pred_label)
# #fp, tp, _ = roc_curve(y_test,pred_label)
# fp, tp, _ = roc_curve(y_test,y_pred[:,1])
# auc = auc(fp,tp)
# fp_li = []
# fp_li.append(fp)
# print(fp_li)

# with open("../result/fp_li(cv%s).txt"%str(cv_cnt), "w") as output:
#      for row in fp_li:
#         output.write(str(row) + '\n')
print('done')
print("cv_cnt =======================",cv_cnt)
print("precision = ",precision)
print("recall = ",recall)
print("f1score = ",f1score)
print("accuracy = ",acc)
# print("auc = ",auc)

# for i in range(len(y_test)):
# 	if y_test[i] == pred_label[i]:
# 		img = array_to_img(X_test[i])
# 		img.save("../../exp_HE/exp_edit2/result_edit/%s/img/true/%s_(T%s_P%s).jpg"%(str(cv_cnt),i,y_test[i],pred_label[i]))
# 	else:
# 		img = array_to_img(X_test[i])
# 		img.save("../../exp_HE/exp_edit2/result_edit/%s/img/false/%s_(T%s_P%s).jpg"%(str(cv_cnt),i,y_test[i],pred_label[i]))
        
# sensitivity = []
# specificity = []
# acc = []
# # dsc = []

# for i in range(len(y_test)):
#     mat = confusion_matrix(y_test,pred_label)
#     if len(mat) == 2:
#         ac = (mat[1,1]+mat[0,0])/(mat[1,0]+mat[1,1]+mat[0,1]+mat[0,0])
#         st = mat[1,1]/(mat[1,0]+mat[1,1])
#         sp = mat[0,0]/(mat[0,1]+mat[0,0])
#         if mat[1,0]+mat[1,1] == 0:
#             specificity.append(sp)
#             acc.append(ac)
#         else:
#             sensitivity.append(st)  
#             specificity.append(sp)
#             acc.append(ac)
#     else:
#         specificity.append(1)
#         acc.append(1)

# # for i in range(len(y_test)):
# #     yt = y_test[i]
# #     yp = pred_label[i]
# #     if np.sum(yt) != 0 and np.sum(yp) != 0:
# #         dice = np.sum(yp[yt == 1])*2.0 / (np.sum(yt) + np.sum(yp))
# #         dsc.append(dice)

# print("complete")      
# print("sensitivity avg : {0:0.4f}".format(np.mean(sensitivity)))
# print("specificity avg : {0:0.4f}".format(np.mean(specificity)))
# # print("dsc avg : {0:0.4f}".format(np.mean(dsc)))
# # print("acc avg : {0:0.4f}".format(np.mean(acc)))

../npy/0
------------------------------
load test images...
------------------------------
(183, 256, 256, 3)
(183, 1)
183/183 [==============================] - 6s 30ms/step
done
cv_cnt ======================= 0
precision =  0.7941176470588235
recall =  0.5684210526315789
f1score =  0.6625766871165645
accuracy =  0.6994535519125683
auc =  <function auc at 0x7f885fba00d0>


In [18]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import r2_score
import cv2
from keras.models import load_model
import os

In [ ]:
# true_list1 = np.load('../npy_edit/0/test_label1_256.npy') #test data mask
# # true_list1 = true_list1.astype('float32')
# # true_list1 = true_list1/255.0
# # true_list1[true_list1 > 0.5] = 1
# # true_list1[true_list1 <= 0.5] = 0
# # print(true_list1.shape)

# true_list2 = np.load('../npy_edit/0/test_label2_256.npy') #test data mask2

# true_list = []
# true_list = np.append(true_list1,true_list2)

# true_list = true_list.astype('float32')
# true_list = true_list/255.0
# true_list[true_list > 0.5] = 1
# true_list[true_list <= 0.5] = 0
# print(true_list.shape)

cv_cnt = 0

mydata = dataProcess(256, 256)
X_test, y_test = mydata.load_test_data(cv_cnt)

pred_list = np.load('../result_edit/npy/test_pred(cv0).npy') #'/home/ubuntu/gcubme4/Workspace/HM_LEE/5th_data/pred/ap_exp1.npy')
pred_list[pred_list > 0.5] = 1
pred_list[pred_list <= 0.5] = 0
# pred_list[pred_list > 127] = 1
# pred_list[pred_list <= 127] = 0

sensitivity = []
specificity = []
acc = []
dsc = []

for i in range(len(y_test)):
    yt = y_test[i].flatten()
    yp = pred_list[i].flatten()
    mat = confusion_matrix(yt,yp)
    if len(mat) == 2:
        ac = (mat[1,1]+mat[0,0])/(mat[1,0]+mat[1,1]+mat[0,1]+mat[0,0])
        st = mat[1,1]/(mat[1,0]+mat[1,1])
        sp = mat[0,0]/(mat[0,1]+mat[0,0])
        if mat[1,0]+mat[1,1] == 0:
            specificity.append(sp)
            acc.append(ac)
        else:
            sensitivity.append(st)  
            specificity.append(sp)
            acc.append(ac)
    else:
        specificity.append(1)
        acc.append(1)

for i in range(len(y_test)):
    yt = y_test[i]
    yp = pred_list[i]
    if np.sum(yt) != 0 and np.sum(yp) != 0:
        dice = np.sum(yp[yt==1])*2.0 / (np.sum(yt) + np.sum(yp))
        dsc.append(dice)

print("complete")      
print("sensitivity avg : {0:0.4f}".format(np.mean(sensitivity)))
print("specificity avg : {0:0.4f}".format(np.mean(specificity)))
print("dsc avg : {0:0.4f}".format(np.mean(dsc)))
print("acc avg : {0:0.4f}".format(np.mean(acc)))

# print("sensitivity min:",np.min(sensitivity))
# print("specificity min:",np.min(specificity))
# print("dsc min:",np.min(dsc))
# print("acc min:",np.min(acc))

# print("sensitivity max:",np.max(sensitivity))
# print("specificity max:",np.max(specificity))
# print("dsc max:",np.max(dsc))
# print("acc max:",np.max(acc))

In [ ]:
import keras
import tensorflow as tf
import h5py
from keras.models import load_model

keras.models.load_model("model_normalize_best(cv0).h5")
np.testing.assert_allclose(model.predict(test_input), reconstructed_model.predict(test_input))


#loss,acc=model.evaluate(test_images,test_labels,verbose=2)
#print("model_acc:{}%".format(100*acc))

In [ ]:
# origin


def sch(epoch):
    if epoch>100:
        return 0.00001
    else:
        return 0.0001
    
def get_nb_files(directory):
    if not os.path.exists(directory):
        return 0
    cnt = 0
    for r, dirs, files in os.walk(directory):
        for dr in dirs:
            cnt += len(glob.glob(os.path.join(r, dr + "/*")))
    return cnt

def add_new_last_layer(base_model, nb_classes):
    x = base_model.layers[-2].output  
    x = GlobalAveragePooling2D()(x)
    #x = Dense(FC_SIZE, activation='relu')(x)
    predictions = Dense(nb_classes, activation='softmax')(x)
    model = Model(input=base_model.input, output=predictions)

    return model

def setup_to_finetune(model):
    for layer in model.layers[:NB_IV3_LAYERS_TO_FREEZE]:
        layer.trainable = False
    for layer in model.layers[NB_IV3_LAYERS_TO_FREEZE:]:
        layer.trainable = True
    multi_model=multi_gpu_model(model,gpus=2)
    multi_model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])

def train():
    for index in range(0,5):
        mydata = dataProcess(256, 256)
        X_train,y_train=mydata.load_train_data(index)
        X_test,y_test=mydata.load_test_data(index)
        print(y_train.shape)
        print(y_test.shape)
        y_train=to_categorical(y_train,2)
        y_test=to_categorical(y_test,2)
    
        # setup model
        base_model = ResNet50(weights='imagenet', include_top=False)
        model = add_new_last_layer(base_model, 2)
        #model.summary()
        #sc=LearningRateScheduler(sch)
        for layer in model.layers[:NB_IV3_LAYERS_TO_FREEZE]:
            layer.trainable = False
        for layer in model.layers[NB_IV3_LAYERS_TO_FREEZE:]:
            layer.trainable = True
        multi_model=multi_gpu_model(model,gpus=2)
        model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

        model_checkpoint = ModelCheckpoint('model_normalize_best(cv%s).h5'%str(index), monitor='val_loss',verbose=1, save_best_only=True, save_weights_only=False)
        sc=LearningRateScheduler(sch)

        model.fit(X_train, y_train, batch_size=40, epochs=300, verbose=1,validation_split=0.2, shuffle=True,callbacks=[model_checkpoint,sc])#1000
        print('predict test data')
        #imgs_mask_test = model.predict(X_test, batch_size=1, verbose=1)
        #np.save('../result/npy/test_pred_1024_nopreprocess.npy', imgs_mask_test)
        model.save("model_normalize_last(cv%s).h5"%str(index))
        
#python classification.py --all_dir ../--nb_epoch 300 --batch_size 8

if __name__=="__main__":
    train()